In [132]:
! pip install optuna  scikit-learn gensim imbalanced-learn xgboost torch

In [133]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report

import numpy as np
from sklearn.metrics import classification_report, accuracy_score

from sklearn.metrics import mean_squared_error
import optuna
from torch.utils.data import WeightedRandomSampler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import time
from collections import Counter
# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm


In [134]:

from google.colab import drive
drive.mount('/content/drive')
BINARY_DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/Atol_with_ROS.csv"

# Record Constants
# Record Constants

#BINARY_DATA_PATH = "../../../data/hAm_with_ROS.csv"

# INPUT_TRAIN_IN = "../../../data/train_in.csv"
# INPUT_TRAIN_OUT = "../../../data/train_out.csv"
# INPUT_TEST_IN = "../../../data/test_in.csv"
# INPUT_TEST_OUT = "../../../data/test_out.csv"
# INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
# INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

# TARGET_MODEL_PATH = '../../webapp/model_files'




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
#Read X Varaibles and Y Varaibles

binary_raw_dataframe =  pd.read_csv(BINARY_DATA_PATH, header=None  , skiprows=1)
print(f"Shape of Dataset  : {binary_raw_dataframe.shape}")


Shape of Dataset  : (309214, 102)


In [136]:
binary_raw_dataframe

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,G,T,C,C,C,A,A,G,T,A,...,C,A,G,G,G,G,G,T,G,NonMoD
1,G,A,A,A,C,C,C,C,A,G,...,A,T,T,T,A,T,A,T,T,NonMoD
2,C,A,G,A,C,A,G,C,C,G,...,G,G,C,A,C,A,G,G,G,NonMoD
3,G,A,A,A,A,G,C,A,C,C,...,A,G,G,T,T,G,G,G,G,NonMoD
4,A,C,A,A,A,A,A,A,G,A,...,G,T,G,A,G,G,G,G,G,NonMoD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309209,T,T,T,C,T,C,T,G,T,A,...,A,A,C,T,G,T,T,T,A,Atol
309210,C,G,T,G,T,C,T,T,A,A,...,T,G,C,T,T,T,T,T,T,Atol
309211,C,A,T,G,A,T,A,C,C,A,...,A,C,A,T,T,A,A,A,A,Atol
309212,A,G,A,A,T,G,T,G,C,A,...,G,C,T,A,T,C,C,C,T,Atol


### Calculate Sequence Positions to extracted from Original Sequence

In [137]:
x_data = binary_raw_dataframe.iloc[:,0:101] # Select data
y_data = binary_raw_dataframe.iloc[:,101] # Select Target

In [138]:
x_data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,G,T,C,C,C,A,A,G,T,A,...,G,C,A,G,G,G,G,G,T,G
1,G,A,A,A,C,C,C,C,A,G,...,G,A,T,T,T,A,T,A,T,T
2,C,A,G,A,C,A,G,C,C,G,...,G,G,G,C,A,C,A,G,G,G
3,G,A,A,A,A,G,C,A,C,C,...,A,A,G,G,T,T,G,G,G,G
4,A,C,A,A,A,A,A,A,G,A,...,G,G,T,G,A,G,G,G,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309209,T,T,T,C,T,C,T,G,T,A,...,A,A,A,C,T,G,T,T,T,A
309210,C,G,T,G,T,C,T,T,A,A,...,C,T,G,C,T,T,T,T,T,T
309211,C,A,T,G,A,T,A,C,C,A,...,G,A,C,A,T,T,A,A,A,A
309212,A,G,A,A,T,G,T,G,C,A,...,T,G,C,T,A,T,C,C,C,T


In [139]:
y_data

0         NonMoD
1         NonMoD
2         NonMoD
3         NonMoD
4         NonMoD
           ...  
309209      Atol
309210      Atol
309211      Atol
309212      Atol
309213      Atol
Name: 101, Length: 309214, dtype: object

In [140]:
concatenated_column= x_data.apply(lambda row: ''.join(map(str, row)), axis=1)
x_data_with_complete_sequence = x_data.assign(Sequence=concatenated_column)

In [141]:
x_data_with_complete_sequence

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,Sequence
0,G,T,C,C,C,A,A,G,T,A,...,C,A,G,G,G,G,G,T,G,GTCCCAAGTACTTTGGATAAAGGGTACTCAACCTGGACTGTGTTAA...
1,G,A,A,A,C,C,C,C,A,G,...,A,T,T,T,A,T,A,T,T,GAAACCCCAGCCAAACTTTCCCAGTCCAGAATACATGATATTTGAT...
2,C,A,G,A,C,A,G,C,C,G,...,G,G,C,A,C,A,G,G,G,CAGACAGCCGGGATAAACCCCTGAAGGATGTGATCATCGCAGACTG...
3,G,A,A,A,A,G,C,A,C,C,...,A,G,G,T,T,G,G,G,G,GAAAAGCACCACCTGTTGCTCACATTCTTCCATGTCAGCTGTGACA...
4,A,C,A,A,A,A,A,A,G,A,...,G,T,G,A,G,G,G,G,G,ACAAAAAAGAAAAACAGTGAAAGAGAGCTGGTGGCAGCCTGGACCC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309209,T,T,T,C,T,C,T,G,T,A,...,A,A,C,T,G,T,T,T,A,TTTCTCTGTAAATCTAAAATCTGATTATGGCACCTTCCCAGTAGTC...
309210,C,G,T,G,T,C,T,T,A,A,...,T,G,C,T,T,T,T,T,T,CGTGTCTTAAAATTTTTTTTTAATGAAAAAATTGTAGCTTTGTCAG...
309211,C,A,T,G,A,T,A,C,C,A,...,A,C,A,T,T,A,A,A,A,CATGATACCAAATATCTGAATTAAACTTATCAAAATACTGTTAATT...
309212,A,G,A,A,T,G,T,G,C,A,...,G,C,T,A,T,C,C,C,T,AGAATGTGCAGGTTTGTTACATAGGTATACACGTGCCATGGTGCTT...


### 3 mer coding

In [142]:
kmer_dict = {}
encoding = 0
k = 3

for sequence in x_data_with_complete_sequence['Sequence']:
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        if kmer not in kmer_dict:
            kmer_dict[kmer] = encoding
            encoding += 1

def encode_with_k_mer_codon(sequence):
    encoded_sequence = []
    for i in range(len(sequence) - k + 1):
        encoded_sequence.append(kmer_dict[sequence[i:i+k]] )

    return np.array(encoded_sequence)

def convertToTensor(x):
    return torch.tensor(x)

### Encode X

In [143]:
X_encoded  =  x_data_with_complete_sequence['Sequence'].apply(encode_with_k_mer_codon)
X_encoded = torch.tensor(X_encoded , dtype =  torch.long)

In [144]:
X_encoded

tensor([[ 0,  1,  2,  ..., 20, 21, 30],
        [35, 18, 24,  ..., 16, 50, 51],
        [48, 36, 28,  ..., 48, 19, 20],
        ...,
        [42, 34, 46,  ..., 17, 18, 18],
        [36, 35, 33,  ...,  1,  2, 26],
        [12, 13, 21,  ..., 30, 29, 31]])

### Encode Y

In [145]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_data = pd.Series(y_data.squeeze())
y_encoded = encoder.fit_transform(y_data)

unique, counts = np.unique(y_encoded, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

{0: 154607, 1: 154607}


In [146]:
print("Generate Train and Split..")
# Train set
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

# Test and Validation set
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


Generate Train and Split..


In [147]:
X_encoded = None
y_encoded = None
x_data_filtered , y_data_filtered = None,None
x_data , y_data = None , None

x_data_with_complete_sequence = None

print("Train Y Count : " ,Counter(y_train))
print("Test Y Count : " ,Counter(y_test))


Train Y Count :  Counter({0: 108253, 1: 108196})
Test Y Count :  Counter({0: 23209, 1: 23174})


### Balance Datset

In [148]:
y_train = torch.tensor(y_train , dtype=torch.long)
y_test = torch.tensor(y_test , dtype=torch.long)
y_valid = torch.tensor(y_valid , dtype=torch.long)

In [149]:
print(X_train.shape)
print(y_train.shape)

torch.Size([216449, 99])
torch.Size([216449])


In [150]:
X_train[1]

tensor([61, 38, 54, 22, 23, 48,  6, 30, 13, 21,  7, 61, 36, 37, 38, 60, 58, 39,
        42, 34, 59, 54, 10, 11, 12, 59, 39, 40, 52, 62,  7, 50, 34, 46, 37, 19,
        47, 49,  2,  2, 55, 56, 20, 21, 31, 45, 23,  4, 33, 43,  1,  2,  2, 55,
        56, 47, 39, 42, 43, 44, 22,  1,  3, 48, 36, 28,  9, 10, 11, 11, 45,  1,
        55, 58, 49, 26, 10, 11, 11, 11, 45, 57, 56, 20, 47, 39,  4, 18, 33, 51,
        32,  8, 41, 48,  6,  0, 23, 40, 52])

In [151]:
hyperparameter = {}
hyperparameter['INPUT_DIMENSION'] = len(kmer_dict) # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide
hyperparameter['HIDDEN_DIMENSION'] = 32
hyperparameter['NO_OF_LAYERS'] = 4
hyperparameter['BATCH_SIZE'] = 32
hyperparameter['OUTPUT_DIMENSION'] = 1
hyperparameter['EMBEDDING_DIMENSION'] = 256 # if you are using Word2Vec Encoding then this should be same as Word2Vec Embedding Dim
hyperparameter['DROP_OUT'] = 0.4
hyperparameter['LEARNING_RATE'] = 0.00001


class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset = RNADataset(X_train, y_train)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

In [152]:
import torch.nn as nn

class RNATransformerModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim, dropout=0.5):
        super(RNATransformerModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        # If batch size first is true then it should be batch size , sequence lenght , embedding dimension
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=8, dim_feedforward=hidden_dim , batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embedding_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.long()
        x_embedded = self.embedding(x)
        #print("Shape of X embedded" , x_embedded.shape)
        x_transformed = self.transformer_encoder(x_embedded)
        x_transformed = x_transformed[:, -1, :]  # taking the last token's output

        output = self.dropout(x_transformed)
        out = self.fc(output)
        return out.squeeze()

In [153]:
# Check data is in correct shape - batch size , sequece len , embedding dimension size
for inputs, labels in train_dataloader:
    print(inputs.shape)
    break

torch.Size([32, 99])


In [154]:

def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.float().to(device)
            outputs = model(inputs)
            if outputs.size() != labels.size(): # skip if batch size mismatch
              continue
            #print(f"Shape of Label : {labels.shape} and output shape : {outputs.shape} ")
            loss = loss_function(outputs, labels)
            running_loss += loss.item()

            predicted = (torch.sigmoid(outputs) > 0.5).float() # Set threshold at 0.5
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            true_labels.extend(labels.cpu().numpy())  # Capture True Labels for Summary Report
            predicted_labels.extend(predicted.cpu().numpy()) # Capture Predicted Labels Lables for Summary Report

    validation_loss = running_loss / len(test_dataloader)
    validation_accuracy = correct / total

    return validation_loss , validation_accuracy , true_labels , predicted_labels



def train_model(model, train_dataloader, test_dataloader, device, epochs, optimizer, loss_function):
    best_val_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        #progress_bar = tqdm(train_dataloader, desc='Epoch {:03d}'.format(epoch + 1), leave=False, disable=False)
        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            if outputs.size() != labels.size(): # skip if batch size mismatch
              continue
            #print(f"Shape of Label : {labels.shape} and output shape : {outputs.shape} ")
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            #progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(inputs))})

        epoch_loss = running_loss / len(train_dataloader)
        val_loss,  validation_accuracy , true_labels , predicted_labels = validate_model(model, test_dataloader, device, loss_function)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Test Accuracy: {validation_accuracy:.4f} , Time Taken : {elapsed_time}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            if no_improvement_count == 10:
                print("No improvement in validation loss for 5 epochs. Training stopped.")
                break



In [155]:
model = RNATransformerModel(input_dim=hyperparameter['INPUT_DIMENSION'],
                            embedding_dim=hyperparameter['EMBEDDING_DIMENSION'],
                            hidden_dim=hyperparameter['HIDDEN_DIMENSION'] ,
                            num_layers = hyperparameter['NO_OF_LAYERS'],
                            output_dim=hyperparameter['OUTPUT_DIMENSION'],
                            dropout=hyperparameter['DROP_OUT'] )


loss_function = nn.BCEWithLogitsLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
optimizer = optim.Adam(model.parameters() ,  lr=hyperparameter['LEARNING_RATE'])

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model Parameters  : " , hyperparameter)

# Train Model with configured Parameter
train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)

Total Number of Parameters for Model Training : 1421729 
Model Parameters  :  {'INPUT_DIMENSION': 67, 'HIDDEN_DIMENSION': 32, 'NO_OF_LAYERS': 4, 'BATCH_SIZE': 32, 'OUTPUT_DIMENSION': 1, 'EMBEDDING_DIMENSION': 256, 'DROP_OUT': 0.4, 'LEARNING_RATE': 1e-05}
Epoch 1, Train Loss: 0.6060, Val Loss: 0.5737, Test Accuracy: 0.6980 , Time Taken : 147.55110454559326
Epoch 2, Train Loss: 0.5762, Val Loss: 0.5692, Test Accuracy: 0.7009 , Time Taken : 145.37003660202026
Epoch 3, Train Loss: 0.5698, Val Loss: 0.5668, Test Accuracy: 0.7044 , Time Taken : 145.66798162460327
Epoch 4, Train Loss: 0.5655, Val Loss: 0.5632, Test Accuracy: 0.7070 , Time Taken : 145.08508944511414
Epoch 5, Train Loss: 0.5622, Val Loss: 0.5618, Test Accuracy: 0.7088 , Time Taken : 145.3620159626007
Epoch 6, Train Loss: 0.5587, Val Loss: 0.5593, Test Accuracy: 0.7095 , Time Taken : 145.5921277999878
Epoch 7, Train Loss: 0.5554, Val Loss: 0.5562, Test Accuracy: 0.7113 , Time Taken : 145.6799671649933
Epoch 8, Train Loss: 0.5520

In [156]:
model

RNATransformerModel(
  (embedding): Embedding(67, 256)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=32, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=32, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=32, bias=True)
        (dropout):

In [157]:
# Evaluate the model on the test dataset
_, final_accuracy, true_labels, predicted_labels = validate_model(model, valid_dataloader,device,loss_function)

# Print the final accuracy
print(f"Final Accuracy: {final_accuracy:.4f}")

# Print the classification summary
print("\n Classification Summary:")
print(classification_report(true_labels, predicted_labels))

Final Accuracy: 0.7179

 Classification Summary:
              precision    recall  f1-score   support

         0.0       0.70      0.76      0.73     23145
         1.0       0.74      0.67      0.70     23237

    accuracy                           0.72     46382
   macro avg       0.72      0.72      0.72     46382
weighted avg       0.72      0.72      0.72     46382

